In [1]:
import tensorflow as tf
from ParticleNet import get_particle_net

2024-07-10 13:37:59.773909: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 13:37:59.838480: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 13:37:59.838510: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 13:37:59.838619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 13:37:59.858104: I tensorflow/core/platform/cpu_feature_g

In [2]:
setting = dict()

# conv_params: list of tuple in the format (K, (C1, C2, C3))
setting["conv_params"] = [
    (16, (64, 64, 64)),
    (16, (128, 128, 128)),
    (16, (256, 256, 256)),
    ]
# conv_pooling: 'average' or 'max'
setting["conv_pooling"] = 'average'
# fc_params: list of tuples in the format (C, drop_rate)
setting["fc_params"] = [(256, 0.1)]

# Model parameters
num_classes = 5
input_shapes = {'points': (100, 3), 'features': (100, 10), 'mask': (100, 1)}


model = get_particle_net(num_classes, {'points': (100, 3), 'features': (100, 10), 'mask': (100, 1)})

2024-07-10 13:38:06.545126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-10 13:38:06.570820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-10 13:38:06.570896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-10 13:38:06.577771: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-10 13:38:06.577825: I tensorflow/compile

In [4]:
import numpy as np
# Generar datos de entrada aleatorios para probar el modelo
points = np.random.rand(2, 100, 3).astype(np.float32)
features = np.random.rand(2, 100, 10).astype(np.float32)
mask = np.random.randint(0, 2, (2, 100, 1)).astype(np.float32)

# Hacer una predicción con el modelo
outputs = model.predict([points, features, mask])

# Verificar la forma de la salida
print("Shape of the output:", outputs.shape)

2024-07-10 13:38:55.087870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:435] Loaded runtime CuDNN library: 8.1.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-07-10 13:38:55.092847: W ./tensorflow/compiler/xla/stream_executor/stream.h:1691] attempting to perform DNN operation using StreamExecutor without DNN support
2024-07-10 13:38:57.243969: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


InternalError: Graph execution error:

Detected at node ParticleNet/ParticleNet_fts_bn/batch_normalization/FusedBatchNormV3 defined at (most recent call last):
  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/olmoa/anaconda3/envs/ctlearn-cluster/lib/python3.10/asyncio/base_events.py", line 595, in 

In [5]:
!pip install pydot

In [4]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names= True, show_layer_activations = True)

AttributeError: module 'pydot' has no attribute 'InvocationException'

In [6]:
model.summary()

Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mask (InputLayer)           [(None, 100, 1)]             0         []                            
                                                                                                  
 MaskCoordShiftLayer (MaskC  (None, 100, 1)               0         ['mask[0][0]']                
 oordShiftLayer)                                                                                  
                                                                                                  
 points (InputLayer)         [(None, 100, 3)]             0         []                            
                                                                                                  
 AddShifttoCoordLayer_layer  (None, 100, 3)               0         ['MaskCoordShiftLaye